# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Deep Learning II</font>

## Word Embedding

Em poucas palavras, Word Embedding transforma texto em números. Essa transformação é necessária porque muitos algoritmos de aprendizado de máquina (incluindo redes profundas) exigem que sua entrada seja vetores de valores contínuos; Eles simplesmente não trabalharão em strings de texto simples.

Assim, uma técnica de modelagem de linguagem natural, como o Word Embedding, é usada para mapear palavras ou frases de um vocabulário para um vetor correspondente de números reais. Além de ser acessível ao processamento por algoritmos de ML, essa representação vetorial possui duas propriedades importantes e vantajosas:

* Dimensionality Reduction - é uma representação mais eficiente
* Contextual Similarity - é uma representação mais expressiva

Se você está familiarizado com a abordagem do Bag of Words (muito usado em Text Mining), você saberá que muitas vezes o resultado é um conjunto de vetores enormes, muito esparsos, onde a dimensionalidade dos vetores que representam cada documento é igual ao tamanho do vocabulário suportado. Word Embedding visa criar uma representação vetorial com um espaço dimensional muito menor.

Word Embedding é usado para análise semântica, para extrair significado do texto a fim de permitir a compreensão da linguagem natural. Para que um modelo de linguagem seja capaz de prever o significado do texto, ele precisa estar ciente da similaridade contextual das palavras. Por exemplo, que tendemos a encontrar palavras de frutas (como maçã ou laranja) em frases onde elas são cultivadas, colhidas, comidas e bebidas, mas não esperamos encontrar esses mesmos conceitos tão perto de, digamos, a palavra avião.

Os vetores criados pela Word Embedding preservam essas semelhanças, de modo que as palavras que ocorrem regularmente nas proximidades do texto também estarão próximas do espaço vetorial. 

Portanto, "O que é Word Embedding?" é um meio de construir uma representação vetorial de baixa dimensionalidade a partir do corpus de texto, que preserva a semelhança contextual das palavras.

## Classificação de Sequências em Texto com LSTMs

A classificação de sequências é um problema de modelagem preditiva em que você possui algumas sequências de dados ao longo do espaço ou do tempo e a tarefa é prever uma categoria para a sequência. O que torna esse problema difícil é que as sequências podem variar em comprimento, ser compostas por um vocabulário muito grande de símbolos de entrada e pode exigir que o modelo aprenda o contexto ou dependências de longo prazo entre símbolos na sequência de entrada. Vejamos como desenvolver um modelo de rede neural recorrente LSTM para problemas de classificação de sequência em Python usando o framework Keras.

O problema que usaremos para demonstrar a aprendizagem de sequências é a classificação do sentimento dos reviews de filmes do IMDB. Podemos rapidamente desenvolver um pequeno modelo LSTM para o problema IMDB e obter uma boa precisão. 

http://www.imdb.com/

In [1]:
import tensorflow as tf
tf.__version__

'2.1.0'

In [2]:
import keras as k
k.__version__

Using TensorFlow backend.


'2.3.1'

In [3]:
# Imports
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [4]:
# Random seed
numpy.random.seed(7)

https://keras.io/datasets/

In [5]:
# Carrega o conjunto de dados, mas mantém apenas as palavras n superiores, zerando o resto
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = top_words)

In [6]:
print(X_train)

[list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 2, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 2, 19, 178, 32])
 list([1, 194, 1153, 194, 2, 78, 228, 5, 6, 1463, 4369, 2, 134, 26, 

In [7]:
print(y_train)

[1 0 0 ... 0 1 0]


In [8]:
# Em seguida, precisamos truncar e ajustar as sequências de entrada para que elas tenham o mesmo comprimento 
# para modelagem. O modelo aprenderá que os valores zero não possuem informações, de modo que as sequências
# não tem o mesmo comprimento em termos de conteúdo, mas os vetores de mesmo comprimento são necessários para 
# executar a computação no Keras.
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen = max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen = max_review_length)

https://keras.io/layers/embeddings/

In [9]:
# Cria o modelo
# Agora podemos definir, compilar e ajustar nosso modelo LSTM. A primeira camada é a camada Embedding, 
# que usa 32 vetores para representar cada palavra. A próxima camada é a camada LSTM com 100
# unidades de memória (neurônios inteligentes). Finalmente, como este é um problema de classificação, 
# usamos uma camada Densa com um único neurônio de saída e ativação sigmóide, que vai gerar previsões igual a 0 ou 1.
# Por se tratar de um problema de classificação binária, usamos como a função de perda (crossentropy binário em Keras). O ADAM conhecido
# O algoritmo de otimização ADAM é usado. O modelo é treinado por apenas 3 épocas pois ele rapidamente pode apresentar 
# overfitting. Um grande tamanho do lote de 64 avaliações é usado para espaçar as atualizações de peso.

embedding_vector_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vector_length, input_length = max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs = 3, batch_size = 64)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
25000/25000 [==============================] - 87s 3ms/step - loss: 0.4826 - accuracy: 0.7604
Epoch 2/3
25000/25000 [==============================] - 86s 3ms/step - loss: 0.3034 - accuracy: 0.8788
Epoch 3/3
25000/25000 [==============================] - 88s 4ms/step - loss: 0.2556 - accuracy: 0.9015


In [10]:
# Avaliação final do modelo
scores = model.evaluate(X_test, y_test, verbose = 0)
print("Acurácia: %.2f%%" % (scores[1]*100))

Acurácia: 84.69%


Você pode ver que este simples modelo LSTM com pouco ajuste atinge resultados próximos do estado da arte para o problema de classificação de texto do IMDB. Importante, este é um modelo que você pode usar para aplicar redes LSTM
para seus próprios problemas de classificação de sequência. Agora, vejamos algumas extensões deste simples modelo.

## Aplicando Regularização com Dropout em LSTMs - Versão 1

In [11]:
# Imports
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

# Random seed 
numpy.random.seed(7)

# Carrega o conjunto de dados, mas mantém apenas as palavras n superiores, zerando o resto
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = top_words)

# Em seguida, precisamos truncar e ajustar as sequências de entrada para que elas tenham o mesmo comprimento 
# para modelagem. 
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen = max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen = max_review_length)

# Cria o modelo
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length = max_review_length))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs = 3, batch_size = 64)

# Avaliação final do modelo
scores = model.evaluate(X_test, y_test, verbose = 0)
print("Acurácia: %.2f%%" % (scores[1]*100))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 500, 32)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
25000/25000 [==============================] - 85s 3ms/step - loss: 0.4858 - accuracy: 0.7552
Epoch

## Aplicando Regularização com Dropout em LSTMs - Versão 2

In [12]:
# Imports
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

# Random seed 
numpy.random.seed(7)

# Carrega o conjunto de dados, mas mantém apenas as palavras n superiores, zerando o resto
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = top_words)

# Em seguida, precisamos truncar e ajustar as sequências de entrada para que elas tenham o mesmo comprimento 
# para modelagem. 
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen = max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen = max_review_length)

# Cria o modelo
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length = max_review_length))
model.add(LSTM(100, dropout = 0.2, recurrent_dropout = 0.2))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs = 3, batch_size = 64)

# Avaliação final do modelo
scores = model.evaluate(X_test, y_test, verbose = 0)
print("Acurácia: %.2f%%" % (scores[1]*100))

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
25000/25000 [==============================] - 87s 3ms/step - loss: 0.5095 - accuracy: 0.7546
Epoch 2/3
25000/25000 [==============================] - 88s 4ms/step - loss: 0.3624 - accuracy: 0.8474
Epoch 3/3
25000/25000 [==============================] - 89s 4ms/step - loss: 0.3242 - accuracy: 0.8686
Acurácia: 84.48%


## LSTMs e CNNs Para Classificação de Sequências

As redes neurais convolucionais são excelentes para aprender a estrutura espacial em dados de entrada. No IMDB os dados de reviews têm uma estrutura espacial unidimensional na sequência de palavras nas revisões e uma CNN poderá escolher características invariantes para o sentimento bom e ruim. Estas características espaciais aprendidas podem então ser aprendidas como sequências por uma camada LSTM. Podemos facilmente adicionar uma CNN unidimensional e as camadas de Max Pooling após a camada Embedding que, em seguida, alimente os recursos consolidados para o LSTM. Podemos usar um pequeno conjunto de 32 recursos com um pequeno filtro de comprimento igual a 3. A camada de pooling pode usar o comprimento padrão de 2 para reduzir a metade o tamanho do mapa de recursos.

In [13]:
# Imports
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.convolutional import Convolution1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

# Random seed 
numpy.random.seed(7)

# Carrega o conjunto de dados, mas mantém apenas as palavras n superiores, zerando o resto
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = top_words)

# Em seguida, precisamos truncar e ajustar as sequências de entrada para que elas tenham o mesmo comprimento 
# para modelagem. 
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

# Cria o modelo
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length = max_review_length))
model.add(Convolution1D(filters = 32, kernel_size = 3, padding = 'same', activation = 'relu'))
model.add(MaxPooling1D(pool_size = 2))
model.add(LSTM(100))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs = 3, batch_size = 64)

# Avaliação final do modelo
scores = model.evaluate(X_test, y_test, verbose = 0)
print("Acurácia: %.2f%%" % (scores[1]*100))

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 500, 32)           3104      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 250, 32)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 216,405
Trainable params: 216,405
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
25000/25000 [==============================] - 45s 2ms/step - loss: 0.4245 - accuracy: 0.7892
Epoch

# Fim